In [2]:
!pip install praw


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.1 MB/s eta 0:00:00a 0:00:01m


In [3]:
import praw
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [4]:
# Cargar las variables de entorno
load_dotenv()
client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')

In [5]:
# Verificación: Obtener información sobre tu cuenta
try:
    user = reddit.user.me()
    print(f"Authenticated as: {user.name}")
except Exception as e:
    print(f"Error during authentication: {e}")

Error during authentication: name 'reddit' is not defined


In [ ]:



# Inicializar el cliente de Reddit
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Lista de subreddits relevantes
subreddits = [
    "cannabis", "weedstocks", "MMJ", "CBD", "eldertrees",
    "psychedelics", "shroomstocks", "psychedelicstudies",
    # Ejemplos de subreddits estatales
    "californialegal", "COents", "Michigents", "FLmedicaltrees"
]

# Palabras clave para filtrar contenido relevante
keywords = [
    "legalization", "medical", "industry", "investment", "stocks",
    "research", "policy", "business", "economy", "finance",
    "psilocybin", "MDMA", "LSD", "ketamine", "ayahuasca"
]

# Tiempo límite para los posts (7 días atrás)
time_limit = datetime.utcnow() - timedelta(days=7)

news_data = {}

for subreddit_name in subreddits:
    print(f"Buscando en r/{subreddit_name}")
    subreddit = reddit.subreddit(subreddit_name)
    news_data[subreddit_name] = []

    # Ajusta el límite según necesidades
    for submission in subreddit.new(limit=100):
        if submission.created_utc < time_limit.timestamp():
            break

        # Verificar si el post contiene palabras clave relevantes
        if any(keyword.lower() in submission.title.lower() or
               (submission.selftext and keyword.lower()
                in submission.selftext.lower())
               for keyword in keywords):
            news_data[subreddit_name].append({
                'title': submission.title,
                'link': f"https://www.reddit.com{submission.permalink}",
                'snippet': submission.selftext[:200] + "..." if submission.selftext else "No self text",
                'published_at': datetime.fromtimestamp(submission.created_utc).isoformat(),
                'score': submission.score,
                'num_comments': submission.num_comments
            })

    print(
        f"Encontrados {len(news_data[subreddit_name])} posts relevantes en r/{subreddit_name}")

# Filtrar subreddits sin posts relevantes
news_data = {subreddit: posts for subreddit,
             posts in news_data.items() if posts}

# Guardar los resultados en un archivo JSON
with open('reddit_cannabis_psychedelics_news.json', 'w') as f:
    json.dump(news_data, f, indent=4)

print("Datos guardados en 'reddit_cannabis_psychedelics_news.json'")